In [11]:
import os
import heapq

def calculate_local_satisfaction(f1, f2):
    """Calculate the Local Robotic Satisfaction between two frameglasses."""
    common = len(f1 & f2)
    f1_diff = len(f1 - f2)
    f2_diff = len(f2 - f1)
    return min(common, f1_diff, f2_diff)

def pair_portraits_optimized(portraits):
    """Pair portraits efficiently using a heap to maximize diversity."""
    portrait_frameglasses = []
    heap = []

    # Precompute all possible pair diversities and store in a heap
    for i in range(len(portraits)):
        for j in range(i + 1, len(portraits)):
            diversity = len(portraits[i][1] | portraits[j][1])  # Union of tags
            heapq.heappush(heap, (-diversity, i, j))  # Max heap using negative diversity

    paired = set()
    while heap and len(paired) < len(portraits):
        _, i, j = heapq.heappop(heap)
        if i not in paired and j not in paired:
            combined_indices = portraits[i][0] + portraits[j][0]
            combined_tags = portraits[i][1] | portraits[j][1]
            portrait_frameglasses.append((combined_indices, combined_tags))
            paired.add(i)
            paired.add(j)

    # Add any leftover single portraits as frameglasses
    for idx in range(len(portraits)):
        if idx not in paired:
            portrait_frameglasses.append(portraits[idx])

    return portrait_frameglasses

def greedy_ordering_optimized(frameglasses):
    """Order frameglasses greedily with reduced complexity."""
    ordered = [frameglasses.pop(0)]  # Start with the first frameglass
    total_score = 0

    while frameglasses:
        # Precompute similarities with the current frameglass
        similarities = [
            (calculate_local_satisfaction(ordered[-1][1], f[1]), idx, f)
            for idx, f in enumerate(frameglasses)
        ]

        # Pick the best next frameglass
        best_score, best_index, best_frameglass = max(similarities)
        ordered.append(best_frameglass)
        total_score += best_score
        frameglasses.pop(best_index)

    return ordered, total_score

def process_paintings(file_path):
    """Read and process paintings from the input file."""
    with open(file_path, 'r') as f:
        lines = f.readlines()

    paintings = []
    for line in lines[1:]:
        parts = line.strip().split()
        paintings.append([parts[0], int(parts[1]), *parts[2:]])

    landscapes = []
    portraits = []

    for idx, painting in enumerate(paintings):
        if painting[0] == 'L':
            landscapes.append(([idx], set(painting[2:])))
        elif painting[0] == 'P':
            portraits.append(([idx], set(painting[2:])))

    # Pair portraits using the optimized method
    portrait_frameglasses = pair_portraits_optimized(portraits)
    landscape_frameglasses = landscapes

    return portrait_frameglasses + landscape_frameglasses

def process_and_output(file_path, output_file_path):
    """Process input and write the output to a file."""
    frameglasses = process_paintings(file_path)
    ordered_frameglasses, max_score = greedy_ordering_optimized(frameglasses)
    num_frameglasses = len(ordered_frameglasses)
    
    # Write output to file
    with open(output_file_path, 'w') as f:
        f.write(f"{max_score}\n")  # Write the total score
        f.write(f"{num_frameglasses}\n")  # Write the number of frameglasses
        for frameglass in ordered_frameglasses:
            f.write(' '.join(map(str, frameglass[0])) + '\n')  # Write indices of the paintings

# Main Function
def main():
    # Define input and output file paths
    input_files = [
       "./Data/11_randomizing_paintings.txt",
    ]
    output_dir = "output_files"
    os.makedirs(output_dir, exist_ok=True)

    for input_file in input_files:
        output_file = os.path.join(output_dir, os.path.basename(input_file).replace(".txt", "_output.txt"))
        process_and_output(input_file, output_file)
        print(f"Processed {input_file}, output saved to {output_file}")

if __name__ == "__main__":
    main()


KeyboardInterrupt: 

In [6]:
import cProfile
import pstats

if __name__ == "__main__":
    profiler = cProfile.Profile()
    profiler.enable()
    main()
    profiler.disable()
    
    # Save the profiling results to a file
    with open("profile_results.txt", "w") as f:
        stats = pstats.Stats(profiler, stream=f)
        stats.strip_dirs()
        stats.sort_stats("cumulative")
        stats.print_stats()


Processed ./Data/10_computable_moments.txt, output saved to output_files\10_computable_moments_output.txt


In [2]:
import os
import heapq
import networkx as nx
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

# Helper Functions
def calculate_local_satisfaction(f1, f2):
    """Calculate the Local Robotic Satisfaction between two frameglasses."""
    common = len(f1 & f2)
    f1_diff = len(f1 - f2)
    f2_diff = len(f2 - f1)
    return min(common, f1_diff, f2_diff)

# Precompute Similarities Using Hashmap
def precompute_similarities(frameglasses):
    """Precompute similarities between all frameglasses."""
    similarity_map = {}
    for i in range(len(frameglasses)):
        for j in range(i + 1, len(frameglasses)):
            similarity = calculate_local_satisfaction(frameglasses[i][1], frameglasses[j][1])
            similarity_map[(i, j)] = similarity
    return similarity_map

# Greedy Ordering with Hashmap
def greedy_ordering_with_hashmap(frameglasses):
    """Order frameglasses greedily using precomputed similarities."""
    similarity_map = precompute_similarities(frameglasses)
    ordered = [frameglasses.pop(0)]  # Start with the first frameglass
    total_score = 0

    while frameglasses:
        # Find the best next frameglass using the hashmap
        best_score, best_index, best_frameglass = max(
            ((similarity_map.get((ordered[-1][0][0], f[0][0]), 0), idx, f)
             for idx, f in enumerate(frameglasses)),
            key=lambda x: x[0]
        )

        ordered.append(best_frameglass)
        total_score += best_score
        frameglasses.pop(best_index)

    return ordered, total_score

# Graph-Based Representation
def build_similarity_graph(frameglasses):
    """Build a graph where nodes are frameglasses and edges are similarities."""
    graph = nx.Graph()
    for i in range(len(frameglasses)):
        graph.add_node(i, frameglass=frameglasses[i])
    for i in range(len(frameglasses)):
        for j in range(i + 1, len(frameglasses)):
            similarity = calculate_local_satisfaction(frameglasses[i][1], frameglasses[j][1])
            graph.add_edge(i, j, weight=similarity)
    return graph

# Optimized Ordering Using a Minimum Spanning Tree (MST)
def order_with_graph_mst(frameglasses):
    """Order frameglasses using a graph and minimum spanning tree."""
    graph = build_similarity_graph(frameglasses)
    mst = nx.minimum_spanning_tree(graph, weight='weight')

    # Traverse the MST to determine the order
    ordered_indices = list(nx.dfs_preorder_nodes(mst))
    ordered = [frameglasses[i] for i in ordered_indices]

    # Compute total score
    total_score = sum(graph[u][v]['weight'] for u, v in zip(ordered_indices, ordered_indices[1:]))
    return ordered, total_score

# Clustering Portraits to Reduce Complexity
def vectorize_tags(tag_sets):
    """Convert tag sets into vectorized format using TF-IDF."""
    tag_strings = [" ".join(tags) for tags in tag_sets]
    vectorizer = TfidfVectorizer()
    return vectorizer.fit_transform(tag_strings)

def cluster_portraits(portraits, num_clusters=10):
    """Cluster portraits into groups based on their tags."""
    tag_sets = [p[1] for p in portraits]
    vectors = vectorize_tags(tag_sets)
    kmeans = KMeans(n_clusters=num_clusters, random_state=42).fit(vectors)
    cluster_labels = kmeans.labels_

    clusters = {i: [] for i in range(num_clusters)}
    for idx, label in enumerate(cluster_labels):
        clusters[label].append(portraits[idx])

    return clusters

def pair_portraits_with_clustering(portraits, num_clusters=10):
    """Pair portraits using clustering to reduce comparisons."""
    clusters = cluster_portraits(portraits, num_clusters)
    paired_frameglasses = []

    for cluster in clusters.values():
        if len(cluster) > 1:
            # Pair within the cluster
            paired_frameglasses.extend(pair_portraits_optimized(cluster))
        elif len(cluster) == 1:
            # Add single portrait directly
            paired_frameglasses.append(cluster[0])

    return paired_frameglasses

# Pairing Portraits Using a Heap
def pair_portraits_optimized(portraits):
    """Pair portraits efficiently using a heap to maximize diversity."""
    portrait_frameglasses = []
    heap = []

    for i in range(len(portraits)):
        for j in range(i + 1, len(portraits)):
            diversity = len(portraits[i][1] | portraits[j][1])
            heapq.heappush(heap, (-diversity, i, j))

    paired = set()
    while heap and len(paired) < len(portraits):
        _, i, j = heapq.heappop(heap)
        if i not in paired and j not in paired:
            combined_indices = portraits[i][0] + portraits[j][0]
            combined_tags = portraits[i][1] | portraits[j][1]
            portrait_frameglasses.append((combined_indices, combined_tags))
            paired.add(i)
            paired.add(j)

    for idx in range(len(portraits)):
        if idx not in paired:
            portrait_frameglasses.append(portraits[idx])

    return portrait_frameglasses

# Painting Processing
def process_paintings(file_path, num_clusters=10):
    """Process paintings with optimized pairing and ordering."""
    with open(file_path, 'r') as f:
        lines = f.readlines()

    paintings = []
    for line in lines[1:]:
        parts = line.strip().split()
        paintings.append([parts[0], int(parts[1]), *parts[2:]])

    landscapes = []
    portraits = []

    for idx, painting in enumerate(paintings):
        if painting[0] == 'L':
            landscapes.append(([idx], set(painting[2:])))
        elif painting[0] == 'P':
            portraits.append(([idx], set(painting[2:])))

    portrait_frameglasses = pair_portraits_with_clustering(portraits, num_clusters)
    landscape_frameglasses = landscapes

    return portrait_frameglasses + landscape_frameglasses

# Optimized Output Process
def process_and_output_advanced(file_path, output_file_path, use_graph=False):
    """Process input and write the output using advanced data structures."""
    frameglasses = process_paintings(file_path)
    if use_graph:
        ordered_frameglasses, max_score = order_with_graph_mst(frameglasses)
    else:
        ordered_frameglasses, max_score = greedy_ordering_with_hashmap(frameglasses)

    num_frameglasses = len(ordered_frameglasses)

    # Write output to file
    with open(output_file_path, 'w') as f:
        f.write(f"{max_score}\n")  # Write the total score
        f.write(f"{num_frameglasses}\n")  # Write the number of frameglasses
        for frameglass in ordered_frameglasses:
            f.write(' '.join(map(str, frameglass[0])) + '\n')

# Main Function
def main():
    input_files = ["./Data/10_computable_moments.txt"]
    output_dir = "output_files"
    os.makedirs(output_dir, exist_ok=True)

    for input_file in input_files:
        output_file = os.path.join(output_dir, os.path.basename(input_file).replace(".txt", "_output_advanced.txt"))
        process_and_output_advanced(input_file, output_file, use_graph=True)
        print(f"Processed {input_file}, output saved to {output_file}")

if __name__ == "__main__":
    main()


Processed ./Data/10_computable_moments.txt, output saved to output_files\10_computable_moments_output_advanced.txt
